In [4]:
import pandas as pd
import numpy as np
import nltk
import gensim

In [2]:
imdb=pd.read_csv('C:/Users/Administrator/Desktop/Term 3/Unstructured Data/datasets/imdb_sentiment.csv')
imdb.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


### Vader (Valence Aware Dictionary and Sentiment Reasoner)

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiment=SentimentIntensityAnalyzer()

C:\Users\Administrator\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [4]:
sentiment.polarity_scores('i love india')

{'neg': 0.0, 'neu': 0.192, 'pos': 0.808, 'compound': 0.6369}

In [5]:
def get_sentiment(text):
    compound=sentiment.polarity_scores(text)['compound']
    if compound>0:
        return 1
    else:
        return 0

In [6]:
imdb['sentiment_vader']=imdb['review'].apply(get_sentiment)

In [7]:
imdb.head()

,review,sentiment,sentiment_vader
0,"A very, very, very slow-moving, aimless movie ...",0,0
1,Not sure who was more lost - the flat characte...,0,0
2,Attempting artiness with black & white and cle...,0,0
3,Very little music or anything to speak of.,0,0
4,The best scene in the movie was when Gerardo i...,1,1


In [8]:
from sklearn.metrics import accuracy_score
accuracy_score(imdb['sentiment'],imdb['sentiment_vader'])

0.7941176470588235

## Topic Modelling

- LSA (Latent Semantic Analysis)
- LDA (Latent Dirichlet Allocation)

- matrix factorization means finding the hidden features of 2 objects. Such as movie and user have genres as a hidden factor.
- can create product catalogue
- gensim expresses it in a compressed format
- compressed data will be 1% of the actual data because the actual data had 99% zeros and no zeros is 1%. It reduces RAM space
- id's are given to the words in this compressed format internally. It saves more space

In [10]:
amazon=pd.read_csv('C:/Users/Administrator/Desktop/Term 3/Unstructured Data/datasets/amazon_reviews_big.csv')
amazon.head()

,asin,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,B000HDJXNA,1,What I recieved is not what is pictured here O...,"12 26, 2012",A29YXBFTD7QUP3,HHA,Buyer be ware,1.356480e+09
1,B006KKS7XQ,5,Excellent unit and a pretty simple install usi...,"09 20, 2013",A3IMTXFYD7CGDN,"Peter W. George ""soyflakeman""",high quality without high price,1.379635e+09
2,B002NP8XJ0,5,"I'm enjoying this keyboard, I'm getting anothe...","08 31, 2010",AXNOW20FQKHVW,B. Hayashi,Superb keyboard + solution for slow wake up an...,1.283213e+09
3,B000EITTLE,4,"Overall, this is a fantastic camera that I'm e...","02 3, 2008",A10KCAK279LO0W,"mmcwatters ""macdadi80""",One qualm: not great in low light,1.201997e+09
4,B006CRXK4S,5,These work very well with mySamsung PN64D7000 ...,"01 28, 2012",A19XXLMZXR764J,S. Garfinkle,"Work great, fit well",1.327709e+09


In [11]:
amazon['reviewText'].head()

0    What I recieved is not what is pictured here O...
1    Excellent unit and a pretty simple install usi...
2    I'm enjoying this keyboard, I'm getting anothe...
3    Overall, this is a fantastic camera that I'm e...
4    These work very well with mySamsung PN64D7000 ...
Name: reviewText, dtype: object

In [12]:
docs=amazon['reviewText'].fillna(' ').str.lower()
docs=docs.str.replace('[^a-z ]','')

In [47]:
docs_clean=[]
stemmer=nltk.stem.PorterStemmer()
stopwords=nltk.corpus.stopwords.words('english')
stopwords.extend(['','work','use','one','get','great'])
for doc in docs:
    words=doc.split(' ')
    words_clean=[stemmer.stem(word) for word in words if word not in stopwords]
    words_clean=[word for word in words_clean if word not in stopwords]
    docs_clean.append(words_clean)

In [14]:
docs_clean[0]

['reciev',
 'pictur',
 'advert',
 'vidio',
 'cabl',
 'job',
 'need',
 'look',
 'high',
 'qualiti',
 'cabl',
 'buy',
 'product',
 'reciev',
 'cheap',
 'thin',
 'flimsi',
 'cabl',
 'gold',
 'plate',
 'anyth',
 'els',
 'claim',
 'bait',
 'switch',
 'mistak',
 'order',
 '',
 '',
 'know',
 'attempt',
 'contact',
 'sender',
 'find',
 'contact',
 'seller',
 'told',
 'contact',
 'amazoncom',
 'amazoncom',
 'good',
 'enough',
 'give',
 '',
 'refund',
 'good',
 'cabl',
 'woth',
 '',
 '',
 'need',
 'run',
 'wife',
 'kareoke',
 'tv',
 'thing',
 'els',
 'would',
 'sent',
 'back',
 'full',
 'refund',
 'im',
 'surpris',
 'item',
 'still',
 'list',
 'high',
 'grade',
 'cabl']

In [48]:
dictionary=gensim.corpora.Dictionary(docs_clean)

In [19]:
len(list(dictionary.values())) 

152950

In [21]:
len(list(dictionary.keys())) #id's

152950

In [27]:
list(zip(dictionary.keys(),dictionary.values()))[0:10]

[(0, ''),
 (1, 'advert'),
 (2, 'amazoncom'),
 (3, 'anyth'),
 (4, 'attempt'),
 (5, 'back'),
 (6, 'bait'),
 (7, 'buy'),
 (8, 'cabl'),
 (9, 'cheap')]

In [28]:
dictionary.doc2bow(docs_clean[0]) # this gives the frequency of an id repeating in the 0th index row of docs_clean

[(0, 5),
 (1, 1),
 (2, 2),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 5),
 (9, 1),
 (10, 1),
 (11, 3),
 (12, 2),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 2),
 (20, 1),
 (21, 2),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 2),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 2),
 (37, 2),
 (38, 1),
 (39, 1),
 (40, 1),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1)]

In [49]:
#creating the input structure for the gensim algorithm
docs_bow=[]
for doc in docs_clean:
    bow=dictionary.doc2bow(doc)
    docs_bow.append(bow)

In [31]:
len(docs_bow[0])

53

In [50]:
lda_model=gensim.models.LdaModel(docs_bow,id2word=dictionary,num_topics=4)

using a coherence model to determine num_topics

In [33]:
lda_model.get_document_topics(docs_bow[1]) #document is mix of 3 topics

[(0, 0.53621256), (1, 0.2111646), (3, 0.24869989)]

In [35]:
lda_model.get_document_topics(docs_bow[3]) #document is mix of 1 topics

[(2, 0.9814274)]

In [36]:
doc2topic=pd.DataFrame(lda_model.get_document_topics(docs_bow[0]),columns=['Topic_no','Prob'])

In [42]:
doc2topic.sort_values('Prob',ascending=False).iloc[0]['Topic_no']

0.0

In [51]:
topics=[]
for bow in docs_bow:
    doc2topic=pd.DataFrame(lda_model.get_document_topics(bow),columns=['Topic_no','Prob'])
    topic=doc2topic.sort_values('Prob',ascending=False)
    topic=topic.iloc[0]['Topic_no']    
    topics.append(topic)

In [52]:
lda_model.print_topics()

[(0,
  '0.021*"case" + 0.018*"drive" + 0.012*"card" + 0.009*"like" + 0.008*"ipad" + 0.007*"fit" + 0.007*"well" + 0.007*"cover" + 0.007*"usb" + 0.007*"laptop"'),
 (1,
  '0.017*"sound" + 0.010*"good" + 0.010*"qualiti" + 0.009*"speaker" + 0.009*"len" + 0.008*"like" + 0.006*"headphon" + 0.006*"look" + 0.006*"price" + 0.006*"realli"'),
 (2,
  '0.009*"connect" + 0.008*"devic" + 0.008*"cabl" + 0.007*"tv" + 0.006*"would" + 0.006*"set" + 0.006*"unit" + 0.006*"time" + 0.005*"like" + 0.005*"problem"'),
 (3,
  '0.024*"camera" + 0.018*"batteri" + 0.010*"charg" + 0.008*"good" + 0.007*"time" + 0.007*"would" + 0.007*"take" + 0.007*"like" + 0.007*"need" + 0.007*"price"')]

### Exercise: Perform topic modelling on ABCnews dataset

In [2]:
news=pd.read_csv('C:/Users/Administrator/Desktop/Term 3/Unstructured Data/datasets/abcnews.csv')
news.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [56]:
news.shape

(1103665, 2)

#### too big records can take 1 lakh records

In [54]:
# news=news.sample(100000) 
# news.shape

(100000, 2)

In [3]:
docs=news['headline_text'].fillna('').str.lower().replace('[^a-z ]','')

In [4]:
docs.head()

0    aba decides against community broadcasting lic...
1       act fire witnesses must be aware of defamation
2       a g calls for infrastructure protection summit
3             air nz staff in aust strike for pay rise
4        air nz strike to affect australian travellers
Name: headline_text, dtype: object

In [5]:
docs_clean=[]
stemmer=nltk.stem.PorterStemmer()
stopwords=nltk.corpus.stopwords.words('english')
for doc in docs:
    words=doc.split(' ')
    words=[stemmer.stem(word) for word in words if word not in stopwords]
    words=[word for word in words if word not in stopwords]
    docs_clean.append(words)  

In [6]:
docs_clean[0]

['aba', 'decid', 'commun', 'broadcast', 'licenc']

In [7]:
dictionary=gensim.corpora.Dictionary(docs_clean)

In [8]:
docs_bow=[]
for docs in docs_clean:
    bow=dictionary.doc2bow(docs)
    docs_bow.append(bow)

In [9]:
lda_model=gensim.models.LdaModel(docs_bow,id2word=dictionary,num_topics=9)

In [10]:
lda_model.print_topics()

[(0,
  '0.025*"wa" + 0.020*"elect" + 0.015*"sex" + 0.013*"test" + 0.012*"say" + 0.011*"turnbul" + 0.010*"child" + 0.010*"find" + 0.010*"labor" + 0.009*"get"'),
 (1,
  '0.017*"state" + 0.013*"fight" + 0.012*"deal" + 0.012*"citi" + 0.011*"feder" + 0.010*"week" + 0.010*"talk" + 0.010*"three" + 0.010*"program" + 0.009*"say"'),
 (2,
  '0.018*"sydney" + 0.016*"sa" + 0.015*"school" + 0.014*"say" + 0.011*"health" + 0.011*"could" + 0.011*"indigen" + 0.011*"chang" + 0.011*"tasmania" + 0.010*"fund"'),
 (3,
  '0.018*"year" + 0.017*"hous" + 0.016*"woman" + 0.016*"show" + 0.013*"live" + 0.013*"market" + 0.012*"power" + 0.012*"miss" + 0.010*"victoria" + 0.009*"share"'),
 (4,
  '0.034*"man" + 0.027*"charg" + 0.027*"court" + 0.023*"melbourn" + 0.020*"murder" + 0.017*"face" + 0.015*"death" + 0.014*"jail" + 0.013*"accus" + 0.012*"drug"'),
 (5,
  '0.019*"open" + 0.019*"brisban" + 0.017*"make" + 0.016*"tasmanian" + 0.016*"hospit" + 0.015*"alleg" + 0.014*"time" + 0.012*"leagu" + 0.011*"run" + 0.009*"lose"')

In [ ]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis=pyLDAvis.gensim.prepare(lda_model,docs_bow,dictionary)

In [ ]:
vis